In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from itertools import permutations
from time import time
import gc
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsRegressor
import os
for dirname, _, filenames in os.walk('C:/Users/mazen/Desktop/m1/chat box/Genome'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/Users/mazen/Desktop/m1/chat box/Genome\genome_scores.csv
C:/Users/mazen/Desktop/m1/chat box/Genome\genome_tags.csv
C:/Users/mazen/Desktop/m1/chat box/Genome\link.csv
C:/Users/mazen/Desktop/m1/chat box/Genome\movie.csv
C:/Users/mazen/Desktop/m1/chat box/Genome\rating.csv
C:/Users/mazen/Desktop/m1/chat box/Genome\tag.csv


In [2]:
seconds = time()
path = 'C:/Users/mazen/Desktop/m1/chat box/Genome/'
rating = pd.read_csv(path + 'rating.csv')
link = pd.read_csv(path + 'link.csv')
genome_tags = pd.read_csv(path + 'genome_tags.csv')
genome_scores = pd.read_csv(path + 'genome_scores.csv')
tag = pd.read_csv(path + 'tag.csv')
movie = pd.read_csv(path + 'movie.csv')
print(time() - seconds)

12.972993850708008


In [3]:
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [4]:
genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [5]:
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [6]:
tag.head(100)

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18
...,...,...,...,...
95,121,36529,Nicolas Cage,2011-03-23 04:04:22
96,121,36529,Nudity (Topless),2011-03-23 04:04:34
97,121,46970,Below R,2011-03-22 04:25:10
98,121,46970,Tarantino,2011-03-22 04:25:02


In [7]:
genome = pd.merge(
    genome_scores,
    genome_tags,
    how = 'left',
    on = 'tagId'
)
genome.head(10)

,movieId,tagId,relevance,tag
0,1,1,0.02500,007
1,1,2,0.02500,007 (series)
2,1,3,0.05775,18th century
3,1,4,0.09675,1920s
4,1,5,0.14675,1930s
5,1,6,0.21700,1950s
6,1,7,0.06700,1960s
7,1,8,0.26275,1970s
8,1,9,0.26200,1980s
9,1,10,0.03200,19th century


In [8]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
dataset = pd.merge(
    rating,
    movie,
    how = 'left',
    on = 'movieId'
)
dataset.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [10]:
dataset = dataset.sample(n = 800000, replace=False, random_state=42)
dataset.shape

(800000, 6)

In [11]:
dataset.head()

,userId,movieId,rating,timestamp,title,genres
17679788,122270,8360,3.5,2012-04-22 01:07:04,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Ro...
7106385,49018,32,2.0,2001-09-11 07:50:36,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
12970708,89527,109374,3.5,2015-01-06 09:26:40,"Grand Budapest Hotel, The (2014)",Comedy|Drama
15426752,106704,1060,3.0,2000-01-22 21:27:57,Swingers (1996),Comedy|Drama
6934678,47791,1732,2.0,2006-01-19 15:48:23,"Big Lebowski, The (1998)",Comedy|Crime


In [12]:
tag_relevance = genome[["movieId", "tag", "relevance"]].groupby(['movieId','tag']).mean()
tag_relevance.sort_values(by="relevance", ascending=False).head(10)
tag_relevance = tag_relevance.sample(n = 800000, replace=False, random_state=42)

In [21]:
## dataset where we have 
tag_relevance.head()

,,relevance
movieId,tag,
27831,atmospheric,0.39475
4129,identity,0.09300
4069,high school,0.10725
5232,author:alan moore,0.04400
5817,mummy,0.00550


In [14]:
### data set by rating ###
movies_by_rating = dataset[["title", "rating"]].groupby('title').mean()
movies_by_rating.sort_values(by="rating", ascending=False).head(10)

,rating
title,
Beastie Boys: Sabotage (1994),5.0
"Human Condition III, The (Ningen no joken III) (1961)",5.0
"All This, and Heaven Too (1940)",5.0
Eichmann (2007),5.0
Tyler Perry's For Colored Girls (2010),5.0
Electrick Children (2012),5.0
Blood on the Sun (1945),5.0
Sword of the Valiant (1984),5.0
Emmanuel's Gift (2005),5.0


In [15]:
#choosing most popular movies 
movie_popularity = dataset['title'].value_counts()
def get_popular_movies(ds, nb):
    ds = ds[ds > nb].index
    return ds
popular_movies = get_popular_movies(movie_popularity,500)

In [16]:
popular_movies_rankings = dataset[dataset["title"].isin(popular_movies)]
popular_movies_rankings.head()

,userId,movieId,rating,timestamp,title,genres
17679788,122270,8360,3.5,2012-04-22 01:07:04,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Ro...
7106385,49018,32,2.0,2001-09-11 07:50:36,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
6934678,47791,1732,2.0,2006-01-19 15:48:23,"Big Lebowski, The (1998)",Comedy|Crime
19116402,132231,3996,5.0,2003-08-04 18:40:10,"Crouching Tiger, Hidden Dragon (Wo hu cang lon...",Action|Drama|Romance
9570821,66226,2302,3.0,2005-05-27 17:37:43,My Cousin Vinny (1992),Comedy


In [17]:
### popular movies by rating ###
movies_by_rating = popular_movies_rankings[["title", "rating"]].groupby('title').mean()
movies_by_rating.sort_values(by="rating", ascending=False).head(10)

,rating
title,
"Shawshank Redemption, The (1994)",4.480962
"Godfather, The (1972)",4.363803
"Usual Suspects, The (1995)",4.354054
Schindler's List (1993),4.341696
Seven Samurai (Shichinin no samurai) (1954),4.331361
"Godfather: Part II, The (1974)",4.298764
Rear Window (1954),4.287449
One Flew Over the Cuckoo's Nest (1975),4.275207
North by Northwest (1959),4.253175


## Experimenting on recommendations by pairs

In [18]:
#most 25 watched movies and then see those that are watched together
movie_popularity = dataset['title'].value_counts().sort_values(ascending = False).head(25).index
dataset_filtered = dataset[dataset['title'].isin(movie_popularity)]

In [19]:
def find_movie_pairs(x):
    pairs = pd.DataFrame(list(permutations(x.values, 2)), columns=['movie_a', 'movie_b'])
    return pairs

movie_combinations = dataset_filtered.groupby('userId')['title'].apply(find_movie_pairs).reset_index(drop=True)
movie_combinations.head(20)

,movie_a,movie_b
0,Apollo 13 (1995),Jurassic Park (1993)
1,Jurassic Park (1993),Apollo 13 (1995)
2,"Shawshank Redemption, The (1994)","Usual Suspects, The (1995)"
3,"Shawshank Redemption, The (1994)",Jurassic Park (1993)
4,"Usual Suspects, The (1995)","Shawshank Redemption, The (1994)"
5,"Usual Suspects, The (1995)",Jurassic Park (1993)
6,Jurassic Park (1993),"Shawshank Redemption, The (1994)"
7,Jurassic Park (1993),"Usual Suspects, The (1995)"
8,Seven (a.k.a. Se7en) (1995),Star Wars: Episode IV - A New Hope (1977)
9,Star Wars: Episode IV - A New Hope (1977),Seven (a.k.a. Se7en) (1995)


In [20]:
combination_counts = movie_combinations.groupby(['movie_a', 'movie_b']).size().reset_index(name = 'size')
combination_counts.sort_values('size', ascending=False, inplace=True)
combination_counts

,movie_a,movie_b,size
179,Forrest Gump (1994),Pulp Fiction (1994),80
295,Pulp Fiction (1994),Forrest Gump (1994),80
177,Forrest Gump (1994),Jurassic Park (1993),77
424,"Silence of the Lambs, The (1991)","Shawshank Redemption, The (1994)",77
400,"Shawshank Redemption, The (1994)","Silence of the Lambs, The (1991)",77
...,...,...,...
269,"Matrix, The (1999)",Dances with Wolves (1990),21
360,Seven (a.k.a. Se7en) (1995),American Beauty (1999),20
14,American Beauty (1999),Seven (a.k.a. Se7en) (1995),20
457,Star Wars: Episode V - The Empire Strikes Back...,Apollo 13 (1995),19
